In [1]:
from oddsapi import OddsApiClient
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime as dt

key = "018c7695b9df48c989f693897e9eeae3"
client = OddsApiClient(api_key=key)


In [2]:
url = "https://api.the-odds-api.com"
ext = "/v3/sports/"
params = {"apiKey": key}
r = requests.get(url=url + ext, params=params)
r.status_code

200

In [ ]:
r.content
sports = json.loads(r.text)
sports['data'][0]
for i in sports['data']:
    print(i['title'], i['active'])

In [28]:

url = "https://api.the-odds-api.com"
req = f"/v3/odds/"
params = {
    "apiKey": key,
    "sport" :'baseball_mlb',
"region" :'us',
'mkt':'h2h',
'oddsFormat': 'american' 
}
odds_req = requests.get(url + req, params=params)

In [29]:
odds = json.loads(odds_req.text)
odds = odds['data']
odds = odds[0]
odds


{'id': 'a050d345eba4fb9557cb2035af72d5e8',
 'sport_key': 'baseball_mlb',
 'sport_nice': 'MLB',
 'teams': ['Atlanta Braves', 'Colorado Rockies'],
 'commence_time': 1654391380,
 'home_team': 'Colorado Rockies',
 'sites': [{'site_key': 'foxbet',
   'site_nice': 'FOX Bet',
   'last_update': 1654399668,
   'odds': {'h2h': [-110, -120]}},
  {'site_key': 'fanduel',
   'site_nice': 'FanDuel',
   'last_update': 1654399682,
   'odds': {'h2h': [112, -142]}},
  {'site_key': 'draftkings',
   'site_nice': 'DraftKings',
   'last_update': 1654399681,
   'odds': {'h2h': [115, -145]}},
  {'site_key': 'bovada',
   'site_nice': 'Bovada',
   'last_update': 1654399714,
   'odds': {'h2h': [-125, -105]}},
  {'site_key': 'pointsbetus',
   'site_nice': 'PointsBet (US)',
   'last_update': 1654399680,
   'odds': {'h2h': [105, -125]}},
  {'site_key': 'barstool',
   'site_nice': 'Barstool Sportsbook',
   'last_update': 1654399711,
   'odds': {'h2h': [115, -148]}},
  {'site_key': 'twinspires',
   'site_nice': 'TwinS

In [50]:
from fractions import Fraction
MY_SPORTSBOOKS = ['Barstool Sportsbook', 'DraftKings', "BetMGM", 'BetRivers', 'FanDuel', 'PointsBet (US)', 'William Hill (Ceasars)']

class Game(object):
    '''
    Object used to organize data from OddsAPI client 
    for processing later
    Inputs:
        data - (dict) data from OddsAPI response. 
        Must include: 'sports_key', 'teams', 'home_team', 
                    'sites'

    '''
    def __init__(self, data):
        self.sport = data['sport_nice']
        self.teams = data['teams']
        self.home = data['home_team']
        for team in self.teams:
            if team != self.home:
                self.away = team
        self.h2h_odds = {}
        for site in data['sites']:
            site_name = site['site_nice']
            self.h2h_odds[site_name] = site['odds']['h2h']
        # self.h2h_odds = pd.DataFrame(self.h2h_odds).T
        # self.h2h_odds.columns = self.teams
        self.start_time = dt.fromtimestamp(int(data['commence_time']))
        self.num_sites = ['sites_count']
    
    def __repr__(self):
        return f"Game object for {self.away} @ {self.home} on {self.start_time}"

    def best_bettable_odds(self):
        '''
        Iterates through the odds and finds the best odds on both sides 
        Currently only supports games with no draws
        '''
        best_team1 = None
        best_team1_site = None
        best_team2 = None
        best_team2_site = None
        for site, odds in self.h2h_odds.items():
            if site in MY_SPORTSBOOKS:
                if not best_team1:
                    best_team1 = odds[0]
                    best_team1_site = site
                    best_team2 = odds[1]
                    best_team2_site = site
                else:
                    if odds[0] > best_team1:
                        best_team1 = odds[0]
                        best_team1_site = site
                    if odds[1] > best_team2:
                        best_team2 = odds[1]
                        best_team2_site = site
        return [(self.teams[0], best_team1_site, best_team1), (self.teams[1], best_team2_site, best_team2)]

    def arb_exists(self):
        best_odds = self.best_bettable_odds()
        decimal_1 = Game.decimal_odds(odds = best_odds[0][-1])
        decimal_2 = Game.decimal_odds(odds = best_odds[1][-1])
        surebet = 1/decimal_1 + 1/decimal_2
        if surebet < 1:
            return True
        else:
            return False

    @staticmethod
    def decimal_odds(odds: int) -> float:
        """
        :param odds: Integer (e.g., -350) or String (e.g., '3/1' or '5/4').
        :return: Float. Odds expressed in Decimal terms.
        """
        if isinstance(odds, float):
            return odds

        elif isinstance(odds, int):
            if odds >= 100:
                return abs(1 + (odds / 100))
            elif odds <= -101 :
                return 100 / abs(odds) + 1
            else:
                return float(odds)

        elif "/" in odds:
            odds = Fraction(odds)
            return round((odds.numerator / odds.denominator) + 1, 2)
        

        

g = Game(odds)

In [38]:
g.h2h_odds

{'FOX Bet': [-110, -120],
 'FanDuel': [112, -142],
 'DraftKings': [115, -145],
 'Bovada': [-125, -105],
 'PointsBet (US)': [105, -125],
 'Barstool Sportsbook': [115, -148],
 'TwinSpires': [104, -132],
 'Unibet': [115, -147],
 'SugarHouse': [104, -132],
 'BetMGM': [105, -135],
 'GTbets': [-113, -113],
 'Betfair': [-115, -115]}

In [51]:
g.best_bettable_odds()
g.arb_exists()

False